In [1]:

# /usr/bin/python
from boto.mturk.connection import MTurkConnection
from boto.mturk.question import ExternalQuestion
from dateutil.parser import *
from boto.mturk.question import HTMLQuestion
import numpy as np
import datetime, hashlib, boto, os, pymongo, json 
import warnings; warnings.simplefilter('ignore')


In [2]:

tyler_id = 'A33F2FVAMGJDGG'


In [3]:

in_progress = {} 


In [4]:

key_info = np.load('../credentials/snail_rootkey.npy').item()


In [5]:
context = 'live'

if context == 'sandbox':
    
    host = 'mechanicalturk.sandbox.amazonaws.com'
    hit_url = 'https://workersandbox.mturk.com/mturk/preview?groupId='
    external_submit = 'https://workersandbox.mturk.com/mturk/externalSubmit'

elif context == 'live':

    print('okay... this one is for real!\n')
    host = 'mechanicalturk.amazonaws.com'
    hit_url = 'https://www.mturk.com/mturk/preview?groupId='
    external_submit = "https://www.mturk.com/mturk/externalSubmit"


okay... this one is for real!



In [6]:

# generate password from the worker id and current time
time_code = datetime.datetime.now()
unique_key = str(time_code)
password = hashlib.md5(unique_key.encode()).hexdigest()


In [7]:
# HIT HTML: instructions, text box, password submit button, success (or error) message, and HIT submit button
question_html_format = """
<html>

  <head>

    <style>
    #main
    {
     margin: 100px auto;
     width: 900px;
     text-align:center;
    }
    </style>

    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'/>
    <script src='https://s3.amazonaws.com/mturk-public/externalHIT_v1.js' type='text/javascript'></script>

    <script>

     function turkGetParam( name, defaultValue )
     {
       var regexS = "[\?&]"+name+"=([^&#]*)";
       var regex = new RegExp( regexS );
       var tmpURL = window.location.href;
       var results = regex.exec( tmpURL );

       if( results == null )
         { return defaultValue }
       else
         { return results[1]}
      }

      var worker_id_from_webpage = turkGetParam( 'workerId', 'NONE' )

      var user_input = '%s'

      function myFunction() {

        password_match = document.getElementById('text_box_input').value === user_input

        if ( password_match )
        {
          document.getElementById("correct_message").style.color = "blue";
          document.getElementById("submit_password").style.display = "none";
          document.getElementById("correct_message").style.display = "block"
          document.getElementById("submitButton").style.display = "inline-block"
        }
        else
        {
          document.getElementById("error_message").style.color = "red";
          document.getElementById("error_message").style.display = "block";}
        }

    </script>

  </head>

  <body>

    <!-- HTML to handle creating the HIT form -->

    <div id='main'>

      <p id="error_message" style='display:none; font-size:20px'>
      You must acccept this HIT for the identity validation step to be successful <br/>
      If you continue getting this message and believe it is an error, send the experimenter an email<br/><br/>
      Otherwise, please return this HIT! <br/>
      The person who this was indended for wont get paid until you do :(
      </p>
      <p id="correct_message" style='display:none; font-size:20px'>
      Thanks so much!<br/>
      We appreciate your time and apologize for the hassle :)<br/>
      Press "Submit" to receive your compensation. It may take up to 24 hours to complete. 
      </p>

      <form name='mturk_form' method='post' id='mturk_form' action='%s'>

        <input type='hidden' value='' name='assignmentId' id='assignmentId'/>
        <!-- set question for subject -->
        <h2>Please enter your password</h2>
        <p><textarea id='text_box_input' name='answer' rows=1 cols=%d></textarea></p>
        <!-- HTML to handle submitting the HIT -->
        <p><input type='submit' id='submitButton' value='Submit' style='display:none'/></p></form>
        <p><input type='submit' id='submit_password' value='PASSWORD SUBMIT' onclick="myFunction()"/></p>

    </div>
    <script language='Javascript'>turkSetAssignmentID();</script>
  </body>

</html>
"""                     %(password, external_submit, len(password)+5)

In [8]:

question_contents = HTMLQuestion(question_html_format, 600)


In [9]:

mtc = MTurkConnection(aws_access_key_id= key_info['AWSAccessKeyId'], 
                      aws_secret_access_key=key_info['AWSSecretKey'], 
                      host=host)


In [10]:
hit_compensation_amount = 0.01

response = mtc.create_hit(question=question_contents,
                          max_assignments=9,
                          title="A compensation HIT for workers with the Stanford Neuro AI lab",
                          description="All other workers will be rejected",
                          keywords="personalized HIT",
                          duration=60*60*60,
                          reward=hit_compensation_amount,
                          approval_delay=None)


In [11]:
if response.status: 
    
    hit_type_id = response[0].HITTypeId
    hit_id = response[0].HITId   
    in_progress[hit_id] = {'bonus_amount':hit_compensation_amount}
    
    print('\n\npassword:', password, '\nurl:',  hit_url + hit_type_id, '\n\n')



password: 0005f2aeb7d360d2413553d9f0d43b10 
url: https://www.mturk.com/mturk/preview?groupId=3Y7FOS1S79CMDPT3KIK19HN0CDN5SG 




In [12]:
# nov 6th
# password: 0005f2aeb7d360d2413553d9f0d43b10 
# url: https://www.mturk.com/mturk/preview?groupId=3Y7FOS1S79CMDPT3KIK19HN0CDN5SG 

In [14]:
in_progress.keys()

dict_keys(['3M4KL7H8KWNZCHL4JQA6K7MZDGA61B'])

In [34]:
# conect to the mongodb
data = json.load(open('../../credentials/auth.json'))
mongo_tunnel = 'mongodb://' + data['user'] + ':' + data['password'] + '@127.0.0.1'
connection = pymongo.MongoClient(mongo_tunnel)
data_base = connection['task_stream']
collection = data_base['match_to_sample']

In [41]:
worker_ids = ['A83OOIGU4606R', 'A2VR9O234UCT9T']

In [16]:
def extract_worker_info(worker_id): 
    
    w_data, bonus_amount = '', ''
    collection_names = ['stimulus_response', 'match_to_sample']
    for i_collection in collection_names: 
        collection = data_base[i_collection]

        if collection.find({'worker_id':worker_id}).count(): 
            
            try: 
                w_data = collection.find({'worker_id':worker_id, "data_type":"worker_feedback"})[0]
                bonus_amount = w_data['total_bonus_earned']
            except: 
                q = collection.find({'worker_id':worker_id,'data_type':'single_trial'})
                bonus_amount = q[q.count()-1]['bonus_earned']

    return w_data, float('%.02f'%bonus_amount)

In [17]:
# _ = [print('\n%s:'%i_worker, extract_worker_info(i_worker)) for i_worker in worker_ids]

In [18]:
hit_id = list(in_progress)[0]

In [20]:
waiting = 1

while waiting: 
    
    result = mtc.get_assignments(hit_id)

    if len(result): 

        worker_id = result[0].WorkerId
        assignment_id = result[0].AssignmentId
        print("\nworker_id: {}, assignment_id: {}, hit_id: {}\n".format(result[0].WorkerId, 
                                                                        result[0].AssignmentId, 
                                                                        result[0].HITId))
        waiting = 0    
        

KeyboardInterrupt: 

In [ ]:
for i_result in range(len(result)): 
    
    worker_id = result[i_result].WorkerId
    assignment_id = result[i_result].AssignmentId

    print(worker_id, assignment_id)

In [93]:
directory = 'compensation_records'
if not os.path.exists(directory): os.makedirs(directory)

if 1: #not os.path.exists(directory + '/' + worker_id + '_' + assignment_id + '.npy'):
    
    print('\nworking on a compensation HIT for %s'%worker_id)
    
    try: 
        print('n trials:', w_data['n_trials'])
        print('duration:', w_data['experiment_duration'])
        print('blocks_completed:', w_data['n_block'])
        print('total_bonus_earned:', w_data['total_bonus_earned'])
        print(w_data['worker_input'])
    except: 
        print("no feedback data for subject\n")

    bonus_amount = bonus_amount 

    MSG = input("Enter a custom message? Pressing enter reverts to default. ")
    if len(MSG) == 0: MSG = "Thanks so much for participating in our study!"
    BONUS = input("Amount to bonus worker? Pressing enter reverts to {} ".format(bonus_amount))
    if len(BONUS) == 0: BONUS = bonus_amount
    converted_bonus = boto.mturk.price.Price(BONUS) 
   
    s = mtc.grant_bonus(worker_id, assignment_id,  converted_bonus, MSG);
    print('success!\n')

    compensation_info = {'time':datetime.datetime.now(), 
                         'worker_id': worker_id, 
                         'message':MSG, 
                         'bonus_amount':bonus_amount,
                         'assignment_id':assignment_id, 
                         'submission_response':{type(s): s}, 
                         'worker_data':w_data}    

    np.save(directory + '/' + worker_id + '_' + assignment_id + '_' + bonus_amount, compensation_info)

else: 

    print('already bonused worker for this compensation HIT')



working on a compensation HIT for ALF9AAZGQP4K5
n trials: 586
duration: 30.15
blocks_completed: 2
total_bonus_earned: 7.694999999999922

Enter a custom message? Pressing enter reverts to default. This is the base rate for the experiment
Amount to bonus worker? Pressing enter reverts to 7.69 0.50
success!

